In [11]:
import numpy as np
from scipy.sparse.linalg import LinearOperator
from scipy.fft import dctn, idctn
import matplotlib.pyplot as plt

# Matrix-free 2D discrete gradient operator

In [8]:
class DiscreteGradientOperatorNeumann(LinearOperator):
    def __init__(self, shape):
        super().__init__(shape=(2 * shape[0] * shape[1], shape[0] * shape[1]), dtype=np.float64)
        self._shape = shape

    def _matvec(self, x):
        x = x.reshape(self._shape)
        gradient_x = np.zeros(self._shape)
        gradient_x[:, :-1] = np.diff(x, axis=1)
        gradient_x[:, -1] = x[:, 0] - x[:, -1] # Neumann boundary condition in x
        gradient_y = np.zeros(self._shape)
        gradient_y[:-1, :] = np.diff(x, axis=0)
        gradient_y[-1, :] = x[0, :] - x[-1, :] # Neumann boundary condition in y
        return np.vstack((gradient_x, gradient_y)).ravel()

    def _rmatvec(self, x):
        x = x.reshape((2, self._shape[0], self._shape[1]))
        adjoint_x = np.zeros(self._shape)
        adjoint_x[:, 0] = x[0, :, -1] - x[0, :, 0] # Neumann boundary condition in x
        adjoint_x[:, 1:] = -np.diff(x[0], axis=1)
        adjoint_y = np.zeros(self._shape)
        adjoint_y[0, :] = x[1, -1, :] - x[1, 0, :] # Neumann boundary condition in y
        adjoint_y[1:, :] = -np.diff(x[1], axis=0)
        return (adjoint_x + adjoint_y).ravel()

# Example usage:
M, N = 10, 3
operator = DiscreteGradientOperatorNeumann((M, N))

# Validation using random vectors
x = np.random.rand(M * N)
y = np.random.rand(2 * M * N)

# Calculate <Ax, y>
Ax_y = np.inner(operator.matvec(x), y)

# Calculate <x, A^*y>
x_Aty = np.inner(x, operator.rmatvec(y))

# Check if the two inner products are approximately equal
tolerance = 1e-10
if np.abs(Ax_y - x_Aty) < tolerance:
    print("Adjoint test passed!")
    print(f"<Ax, y> = {Ax_y}, <x, A^*y> = {x_Aty}")
else:
    print("Adjoint test failed.")
    print(f"<Ax, y> = {Ax_y}, <x, A^*y> = {x_Aty}")


Adjoint test passed!
<Ax, y> = -0.3624105135683702, <x, A^*y> = -0.36241051356836995


# Is it diagonalized by DCT?

In [9]:
M, N = 10, 3
operator = DiscreteGradientOperatorNeumann((M, N))

In [53]:
# Draw random vector
v = 3 + np.random.normal(size=M*N)
z = idctn( operator.rmatvec( operator.matvec( dctn( v, norm="ortho", orthogonalize=True  ) ) ), norm="ortho", orthogonalize=True ) / v
np.sort(z)

array([-0.31883193, -0.03342873,  0.14460121,  0.86005666,  1.77204654,
        2.3057544 ,  2.40110244,  3.08025545,  3.13760329,  3.57589473,
        3.64458657,  3.74824413,  3.76422006,  4.03737227,  4.31308116,
        4.347854  ,  4.49174667,  4.671976  ,  4.71402824,  5.32154723,
        5.36069431,  5.70181667,  5.82386048,  5.85338999,  5.88949612,
        5.99510127,  6.03239044,  6.20816333,  6.60591882,  7.54915684])

In [62]:
# Draw random vector
v = 3 + np.random.normal(size=M*N)
z = dctn( operator.rmatvec( operator.matvec( idctn( v, norm="ortho", orthogonalize=True  ) ) ) , norm="ortho", orthogonalize=True) / v
np.sort(z)

array([-3.03331742e-01,  4.20462762e-16,  2.23380331e+00,  2.53348560e+00,
        3.00000000e+00,  3.02036426e+00,  3.22908881e+00,  3.31447913e+00,
        3.51870916e+00,  3.63995870e+00,  3.67887885e+00,  3.92898268e+00,
        3.98052015e+00,  4.67065400e+00,  4.89312254e+00,  4.93844775e+00,
        5.33347302e+00,  5.53869560e+00,  5.61531297e+00,  5.66642660e+00,
        5.99144069e+00,  6.36697169e+00,  6.61135517e+00,  6.97804682e+00,
        7.00000000e+00,  7.01500705e+00,  7.07668680e+00,  7.09968270e+00,
        7.09978987e+00,  8.23059237e+00])

In [80]:
# Draw random vector
v = 3 + np.random.normal(size=M*N)
z = dctn( operator.rmatvec( operator.matvec( idctn( v, norm="ortho", orthogonalize=False  ) ) ) , norm="ortho", orthogonalize=False) / v
np.sort(z)

array([5.98266942e-16, 5.73275503e-01, 2.04332331e+00, 2.69422960e+00,
       2.81479006e+00, 3.00000000e+00, 3.15946802e+00, 3.29760354e+00,
       3.61506284e+00, 3.75646308e+00, 3.83666930e+00, 4.12111325e+00,
       4.46575522e+00, 4.71439938e+00, 5.16618892e+00, 5.31959608e+00,
       5.33647866e+00, 5.40845719e+00, 5.41071415e+00, 5.83588130e+00,
       5.90429053e+00, 6.08360072e+00, 6.41201449e+00, 6.84322204e+00,
       6.86748849e+00, 6.92862570e+00, 6.94482001e+00, 7.00000000e+00,
       7.04911010e+00, 8.36252980e+00])

In [87]:
# Draw random vector
v = 3 + np.random.normal(size=M*N)
z = idctn( operator.rmatvec( operator.matvec( dctn( v, norm="ortho", orthogonalize=False  ) ) ), norm="ortho", orthogonalize=False ) / v
np.sort(z)

array([-22.72478243,  -2.58571848,  -0.80724228,  -0.74607839,
        -0.32365331,   0.38763778,   1.8942192 ,   2.06030042,
         2.1071344 ,   2.92818543,   3.34623409,   3.57200636,
         4.0125864 ,   4.19606801,   4.44411001,   4.48482351,
         4.64520208,   4.99467894,   5.13565393,   5.69758115,
         5.90385563,   6.1711313 ,   6.25341937,   6.84041996,
         6.85271766,   7.42399905,   7.56440257,   7.75678349,
         8.32822613,   9.50915277])